In [3]:
!pip install schedule

In [4]:
from selenium import webdriver
import schedule
import time
import sqlite3
from bs4 import BeautifulSoup


def job():

    driver = webdriver.Chrome('./chromedriver.exe')
    conn = sqlite3.connect('./db.stock')
    c = conn.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS article (id INTEGER PRIMARY KEY AUTOINCREMENT, date TEXT, time TEXT, title TEXT, content TEXT, press TEXT , href TEXT, stock TEXT, posi_nega TEXT)")

    uri = 'https://finance.naver.com/news/market_special.nhn?&page='
#     fnews = []

    for page in range(1, 3):
        target = uri+str(page)
        driver.get(target)

        try:
            for n in range(1, 24):
    #             time.sleep(2)
                driver.implicitly_wait(5)
                try: 
                    driver.find_element_by_xpath('//*[@id="contentarea_left"]/div[2]/table/tbody/tr[%s]/td[1]/a' %n).click()
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')

                    presses = soup.select('span.press > img')
                    press = presses[0]['title']

                    datas = soup.select('div.article_info > h3')
                    title = datas[0].text.strip()

                    dates = soup.select('div.article_sponsor > span')
                    date = dates[0].text.strip()

                    contents = soup.select('div#content')
                    content1 = contents[0]
                    for tag in content1.find_all(['li', 'h3']):
                        tag.replace_with('')
                        content = content1.text.strip()

                    driver.back()

                    html = driver.page_source
                    soup1 = BeautifulSoup(html, 'html.parser')

                    times = soup1.select('td.wdate')
                    time1 = times[int(n)-1]
                    time = time1.text.strip()[-5:]

                    hrefs = soup1.select('td.publicSubject > a')
                    href1 = hrefs[int(n)-1]
                    href = 'https://finance.naver.com'+ href1['href']
                    
#                     a = [date , time , press , title, content, href]
#                     fnews.append(a)

                    c.execute("INSERT INTO article( date , time , title, content, press, href, stock , posi_nega ) VALUES(?,?,?,?,?,?,?,?)",(date, time, title, content, press, href,'stock','posi_nega'))

                except:
                    pass

        except:
            pass  

    conn.commit()
    c.close()

#     print(fnews)    
    
schedule.every().day.at("06:00").do(job) 

while True:
    schedule.run_pending()
    time.sleep(5)


[['2021-08-19', '15:55', '연합뉴스', '크래프톤 5% 급등…장중 공모가 회복(종합)', "크래프톤, 코스피 상장역대 2위의 공모 규모에도 청약 흥행에서 참패한 게임업체 크래프톤이 유가증권시장에 상장한 지난 10일 오후 서울 여의도 한국거래소 외벽에 크래프톤 상장 축하 현수막이 걸려 있는 모습.[연합뉴스 자료사진](서울=연합뉴스) 박진형 기자 = 상장 이후 주가가 공모가 밑으로 떨어졌던 게임업체 크래프톤이 19일 급등, 장중 한때 공모가를 회복했다.    이날 유가증권시장에서 크래프톤은 전날보다 5.01% 상승한 49만2천500에 거래를 마쳤다.    지난 10일 상장한 크래프톤은 이날 장중 한때 49만8천500원까지 뛰어올라 7거래일만에 공모가(49만8천원)를 넘어서기도 했다.    앞서 지난 17일 크래프톤은 인도 시장에 내놓은 모바일 게임 '배틀그라운드 모바일 인도'가 출시 44일 만에 누적 다운로드 5천만회를 돌파했다고 발표했다    이 게임은 지난달 2일 구글플레이 출시 이후 일주일 만에 누적 이용자수 3천400만명, 일일 최대 이용자수 1천600만명, 최대 동시 접속자수 240만명을 각각 기록했다.     크래프톤은 또 2분기 연결기준 영업이익이 1천742억원으로 엔씨소프트, 넷마블, 넥슨 등 이른바 '3N' 게임사들을 추월했다.    jhpark@yna.co.kr", 'https://finance.naver.com/news/market_special_read.nhn?article_id=0012608954&office_id=001&type=title&keyword='], ['2021-08-19', '15:50', '연합뉴스', "롯데렌탈, 상장일 공모가 하회…브레인즈컴퍼니 '따상'(종합)", "롯데렌탈 공모주 청약NH투자증권 영업점에 롯데렌탈 공모주 청약을 알리는 안내판이 서 있다. [NH투자증권 제공. 재판매 및 DB금지](서울=연합뉴스) 김태종 기자 = 19일 유가증권시장에 상장한 롯데렌탈이 공모가 아래에서 첫 거래를 마감했다

KeyboardInterrupt: 